### Prerequsite - run FinalTestPCR.ipynb first

In [8]:
import joblib
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# Install xlrd package
%pip install xlrd

all_df = pd.read_excel('TestDatasetExample.xls', index_col=False)
IDs = all_df['ID']
all_df.drop('ID', axis=1, inplace=True)
all_df.head()



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


,Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,HistologyType,LNStatus,TumourStage,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,56.881588,0,0,0,1,3,3,999,0,2,...,0.194591,0.194591,2.846439,0.001281,4.168474e+06,131.044541,0.002335,0.109755,0.013383,0.002051
1,60.000000,0,0,1,0,2,1,1,0,3,...,0.309999,0.309996,2.975317,0.007253,1.736585e+05,23.967478,0.011285,0.055890,0.003163,0.009553
2,58.234086,0,0,0,1,3,3,1,1,4,...,0.328377,0.328377,3.785966,0.003185,3.607821e+06,223.279556,0.001334,0.101628,0.010844,0.001194


### Fill train set with Classification result

In [9]:
# Add the prediction to the dataframe
df_clf = pd.read_csv('results/classification_outcome.csv')
all_df['pCR (outcome)'] = df_clf['pCR (outcome)']
all_df.head()

,Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,HistologyType,LNStatus,TumourStage,...,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength,pCR (outcome)
0,56.881588,0,0,0,1,3,3,999,0,2,...,0.194591,2.846439,0.001281,4.168474e+06,131.044541,0.002335,0.109755,0.013383,0.002051,0
1,60.000000,0,0,1,0,2,1,1,0,3,...,0.309996,2.975317,0.007253,1.736585e+05,23.967478,0.011285,0.055890,0.003163,0.009553,1
2,58.234086,0,0,0,1,3,3,1,1,4,...,0.328377,3.785966,0.003185,3.607821e+06,223.279556,0.001334,0.101628,0.010844,0.001194,0


# Part 2: RFS Regression

### Pre-process for Regression

In [10]:

from sklearn.impute import SimpleImputer
import pickle

# load gene clf feature names
import json
keep_feat_names = []
with open('gene_clf_selected_features.json', 'rb') as f:
  keep_feat_names = json.load(f)


if 999 not in all_df['Gene'].values:
  print("no missing Gene - skipping gene impute")
else:
  # replace missing gene with classification result
  # see train_gene_classifier.ipynb for more details
  with open('log_reg_gene_classifier.pkl', 'rb') as f:
    log_res_clf = pickle.load(f)

    # rebuild prediction df
    gene_impute_df = all_df.copy()

    temp_X = gene_impute_df.drop(['pCR (outcome)'], axis=1)
    y = gene_impute_df['Gene']

    print("before impute:") 
    print(gene_impute_df['Gene'].value_counts())

    keep_df = gene_impute_df[keep_feat_names]
    replace_index = keep_df[keep_df['Gene'] == 999].index

    # get prediction on missing gene
    target = gene_impute_df.loc[replace_index, keep_feat_names]
    target.drop('Gene', axis=1, inplace=True)

    print("target shape:", target.shape)

    pred = log_res_clf.predict(target)
    gene_impute_df.loc[replace_index, 'Gene'] = pred

    print("after impute:") 
    print(gene_impute_df['Gene'].value_counts())

    # assign back to all_df
    all_df['Gene'] = gene_impute_df['Gene']

  # Replace missing values with median of the column
  imputer = SimpleImputer(strategy="median", missing_values=999)
  all_df[:] = imputer.fit_transform(all_df)

before impute:
Gene
0      1
999    1
1      1
Name: count, dtype: int64
target shape: (1, 14)
after impute:
Gene
0    2
1    1
Name: count, dtype: int64


/Users/stephen/Library/Caches/pypoetry/virtualenvs/labs-env-NHQiDXzZ-py3.12/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Xs = scaler.fit_transform(all_df)

In [12]:
### Feature Selection and Dimensionality Reduction strategy:
#
# 1. Keep all non mri columns
# 2. Select the mri_pca_2MRI features and apply PCA
# 3. Combine the two sets of features

non_mri_indicies = list(range(13))
print(all_df.columns[non_mri_indicies])

# Select the MRI features
mri_indices = list(range(13, Xs.shape[1]))
print(all_df.columns[mri_indices])

# Apply PCA to the MRI features
from sklearn.decomposition import PCA

pca = PCA(n_components=1)
X_mri = Xs[:, mri_indices]
X_mri_pca = pca.fit_transform(X_mri)

# Combine the two sets of features
non_mri_feats = Xs[:, non_mri_indicies]
Xs = np.hstack([non_mri_feats, X_mri_pca])

Index(['Age', 'ER', 'PgR', 'HER2', 'TrippleNegative', 'ChemoGrade',
       'Proliferation', 'HistologyType', 'LNStatus', 'TumourStage', 'Gene',
       'original_shape_Elongation', 'original_shape_Flatness'],
      dtype='object')
Index(['original_shape_LeastAxisLength', 'original_shape_MajorAxisLength',
       'original_shape_Maximum2DDiameterColumn',
       'original_shape_Maximum2DDiameterRow',
       'original_shape_Maximum2DDiameterSlice',
       'original_shape_Maximum3DDiameter', 'original_shape_MeshVolume',
       'original_shape_MinorAxisLength', 'original_shape_Sphericity',
       'original_shape_SurfaceArea',
       ...
       'original_glszm_SmallAreaLowGrayLevelEmphasis',
       'original_glszm_ZoneEntropy', 'original_glszm_ZonePercentage',
       'original_glszm_ZoneVariance', 'original_ngtdm_Busyness',
       'original_ngtdm_Coarseness', 'original_ngtdm_Complexity',
       'original_ngtdm_Contrast', 'original_ngtdm_Strength', 'pCR (outcome)'],
      dtype='object', length

### Load Model and Predict

In [13]:
with open('rfs_regression_model.pkl', 'rb') as f:
  rfs_model = pickle.load(f)

  # predict RFS
  rfs_pred = rfs_model.predict(Xs)
  print(rfs_pred)
  
  all_df['RelapseFreeSurvival (outcome)'] = rfs_pred



[51.99880394 69.58622199 47.03886763]


In [14]:
### Save to results to csv
rfs_coutcome = all_df['RelapseFreeSurvival (outcome)']

reg_outcome = pd.DataFrame({'ID': IDs,'RelapseFreeSurvival (outcome)': rfs_coutcome,})

reg_outcome.to_csv('results/regression_outcome.csv', index=False)